# Demonstração da segmentação de imagens com Mask R-CNN

Neste laboratório, você verá como usar um modelo [Mask R-CNN] (https://arxiv.org/abs/1703.06870) do Tensorflow Hub para detecção de objetos e segmentação de instâncias. Isso significa que, além das caixas delimitadoras, o modelo também é capaz de prever máscaras de segmentação para cada instância de uma classe na imagem. Você já encontrou a maioria dos comandos aqui quando trabalhou com a API Object Dection e verá como pode usá-la com modelos de segmentação de instância. Vamos começar!

*Observação: você deve usar uma TPU para esse laboratório devido aos requisitos de processamento desse modelo. Já o ativamos para você, mas se for usá-lo em outro colab, poderá alterar o tempo de execução em `Runtime --> Change runtime type` e selecionar `TPU`.*

## Instalação

Conforme mencionado, você usará a [API de detecção de objetos do Tensorflow 2](https://github.com/tensorflow/models/tree/master/research/object_detection). Você pode fazer isso clonando o [Tensorflow Model Garden](https://github.com/tensorflow/models) e instalando os pacotes de detecção de objetos, como fez anteriormente.

In [1]:
# Clonar o repositório de modelos do Tensorflow
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3909, done.
remote: Counting objects: 100% (3909/3909), done.
remote: Compressing objects: 100% (3012/3012), done.
remote: Total 3909 (delta 1130), reused 2009 (delta 844), pack-reused 0
Receiving objects: 100% (3909/3909), 49.65 MiB | 26.80 MiB/s, done.
Resolving deltas: 100% (1130/1130), done.


In [2]:
# Compilar os buffers de protocolo da API de detecção de objetos
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

In [3]:
%%writefile models/research/setup.py

import os
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
    'tf-models-official==2.7.0',
    'tensorflow_io'
]

setup(
    name='object_detection',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    include_package_data=True,
    packages=(
        [p for p in find_packages() if p.startswith('object_detection')] +
        find_packages(where=os.path.join('.', 'slim'))),
    package_dir={
        'datasets': os.path.join('slim', 'datasets'),
        'nets': os.path.join('slim', 'nets'),
        'preprocessing': os.path.join('slim', 'preprocessing'),
        'deployment': os.path.join('slim', 'deployment'),
        'scripts': os.path.join('slim', 'scripts'),
    },
    description='Tensorflow Object Detection Library',
    python_requires='>3.6',
)

Writing models/research/setup.py


In [4]:
# Execute o script de configuração que você acabou de escrever
!python -m pip install models/research

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./models/research
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 47.9 MB/s eta 0:00:00
  Created wheel for object-detection: filename=object_detection-0.1-

## Importar bibliotecas

In [6]:
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from six import BytesIO
from PIL import Image
from six.moves.urllib.request import urlopen

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

tf.get_logger().setLevel('ERROR')

%matplotlib inline

## Utilitários

Por conveniência, você usará uma função para converter uma imagem em uma matriz numpy. Você pode passar um caminho relativo para uma imagem (por exemplo, para um diretório local) ou um URL. Você pode ver isso no dicionário `TEST_IMAGES` abaixo. Alguns caminhos apontam para imagens de teste que vêm com o pacote da API (por exemplo, `Beach`), enquanto outros são URLs que apontam para imagens on-line (por exemplo, `Street`).

In [7]:
def load_image_into_numpy_array(path):
  """Carrega uma imagem de um arquivo em uma matriz numpy.

  Coloca a imagem em uma matriz numpy para alimentar o gráfico do tensorflow.
  Observe que, por convenção, nós a colocamos em uma matriz numpy com a forma
  (altura, largura, canais), onde canais=3 para RGB.

  Args:
    path: o caminho do arquivo para a imagem

  Retorna:
    matriz numpy uint8 com formato (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = (image.size)
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


# dicionário com tags de imagem como chaves e caminhos de imagem como valores
TEST_IMAGES = {
  'Beach' : 'models/research/object_detection/test_images/image2.jpg',
  'Dogs' : 'models/research/object_detection/test_images/image1.jpg',
  # By Américo Toledano, Source: https://commons.wikimedia.org/wiki/File:Biblioteca_Maim%C3%B3nides,_Campus_Universitario_de_Rabanales_007.jpg
  'Phones' : 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/0d/Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg/1024px-Biblioteca_Maim%C3%B3nides%2C_Campus_Universitario_de_Rabanales_007.jpg',
  # By 663highland, Source: https://commons.wikimedia.org/wiki/File:Kitano_Street_Kobe01s5s4110.jpg
  'Street' : 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/Kitano_Street_Kobe01s5s4110.jpg/2560px-Kitano_Street_Kobe01s5s4110.jpg'
}

## Load the Model

Tensorflow Hub provides a Mask-RCNN model that is built with the Object Detection API. You can read about the details [here](https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1). Let's first load the model and see how to use it for inference in the next section.

In [9]:
model_display_name = 'Mask R-CNN Inception ResNet V2 1024x1024'
model_handle = 'https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1'

print('Selected model:'+ model_display_name)
print('Model Handle at TensorFlow Hub: {}'.format(model_handle))

Selected model:Mask R-CNN Inception ResNet V2 1024x1024
Model Handle at TensorFlow Hub: https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1


In [10]:
# This will take 10 to 15 minutes to finish
print('loading model...')
hub_model = hub.load(model_handle)
print('model loaded!')

loading model...
model loaded!


## Inferência

Você usará o modelo que acabou de carregar para fazer a segmentação de instância em uma imagem. Primeiro, escolha uma das imagens de teste que você especificou anteriormente e carregue-a em uma matriz numpy.

In [11]:
# Escolha uma e use-a como chave para TEST_IMAGES abaixo:
# ['Beach', 'Street', 'Dogs','Phones']

image_path = TEST_IMAGES['Street']

image_np = load_image_into_numpy_array(image_path)

plt.figure(figsize=(24,32))
plt.imshow(image_np[0])
plt.show()

Output hidden; open in https://colab.research.google.com to view.

Você pode executar a inferência simplesmente passando a matriz numpy de uma *única* imagem para o modelo. Observe que esse modelo não oferece suporte a lotes. Como você viu nos notebooks da Semana 2, isso produzirá um dicionário contendo os resultados. Eles são descritos na seção `Outputs` da [documentação](https://tfhub.dev/tensorflow/mask_rcnn/inception_resnet_v2_1024x1024/1)

In [12]:
# executar inferência
results = hub_model(image_np)

# os valores de saída são tensores e só precisamos do parâmetro numpy()
# quando visualizamos os resultados
result = {key:value.numpy() for key,value in results.items()}

# imprimir as chaves
for key in result.keys():
  print(key)

num_proposals
rpn_box_predictor_features
num_detections
proposal_boxes_normalized
detection_classes
refined_box_encodings
box_classifier_features
raw_detection_boxes
rpn_features_to_crop
final_anchors
detection_boxes
rpn_box_encodings
detection_anchor_indices
class_predictions_with_background
proposal_boxes
raw_detection_scores
rpn_objectness_predictions_with_background
mask_predictions
detection_multiclass_scores
anchors
detection_masks
detection_scores
image_shape


## Visualizando os resultados

Agora você pode plotar os resultados na imagem original. Primeiro, você precisa criar o dicionário `category_index` que conterá os IDs e nomes das classes. O modelo foi treinado no conjunto de dados [COCO2017](https://cocodataset.org/) e o pacote API tem os rótulos salvos em um formato diferente (ou seja, `mscoco_label_map.pbtxt`). Você pode usar a função de utilitário interno [create_category_index_from_labelmap](https://github.com/tensorflow/models/blob/5ee7a4627edcbbaaeb8a564d690b5f1bc498a0d7/research/object_detection/utils/label_map_util.py#L313) para converter isso no formato de dicionário necessário.

In [13]:
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# sample output
print(category_index[1])
print(category_index[2])
print(category_index[4])

{'id': 1, 'name': 'person'}
{'id': 2, 'name': 'bicycle'}
{'id': 4, 'name': 'motorcycle'}


Em seguida, você pré-processará as máscaras e, finalmente, plotará os resultados.

* O dicionário de resultados contém uma chave `detection_masks` que contém máscaras de segmentação para cada caixa. Isso será convertido primeiro em máscaras que serão sobrepostas ao tamanho total da imagem.
* Você também selecionará os valores de pixel da máscara que estão acima de um determinado limite. Escolhemos um valor de `0,6`, mas sinta-se à vontade para modificá-lo e ver quais resultados você obterá. Se você escolher um valor mais baixo, provavelmente notará pixels de máscara que estão fora do objeto.
* Como você já viu anteriormente, é possível usar `visualize_boxes_and_labels_on_image_array()` para plotar os resultados na imagem. A diferença desta vez é o parâmetro `instance_masks` e você passará as caixas de detecção reenquadradas para ver as máscaras de segmentação na imagem.

Você pode ver como tudo isso é tratado no código abaixo.

In [14]:
# Manipular modelos com máscaras:
label_id_offset = 0
image_np_with_mask = image_np.copy()

if 'detection_masks' in result:

  # converter np.arrays em tensores
  detection_masks = tf.convert_to_tensor(result['detection_masks'][0])
  detection_boxes = tf.convert_to_tensor(result['detection_boxes'][0])

  # Reenquadre a máscara da caixa delimitadora de acordo com o tamanho da imagem.
  detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes,
              image_np.shape[1], image_np.shape[2])

  # valores de pixel da máscara de filtro que estão acima de um limite especificado
  detection_masks_reframed = tf.cast(detection_masks_reframed > 0.6,
                                      tf.uint8)

  # obter a matriz numérica
  result['detection_masks_reframed'] = detection_masks_reframed.numpy()

# Sobrepor caixas rotuladas e máscaras de segmentação na imagem
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_mask[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=100,
      min_score_thresh=.70,
      agnostic_mode=False,
      instance_masks=result.get('detection_masks_reframed', None),
      line_thickness=8)

plt.figure(figsize=(24,32))
plt.imshow(image_np_with_mask[0])
plt.show()

Output hidden; open in https://colab.research.google.com to view.